In [1]:
import numpy as np
from numpy import random
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
from matplotlib import colors
import vpython as vp
import time
%matplotlib ipympl

<IPython.core.display.Javascript object>

# Simulated annealing

## Anneal

In [2]:
def MC_t(J, H, T, S=None, n=20, nMC=1000, debug=False):
    """
    J: exchange interaction between nearest neigbor spins
    H: magnetic field: 1 by 3
    T: temperature in unit of J
    S: spin directions, num_of_spins by 3
    n: num_of_spins
    nMC：num of MC steps per spin
    """
    # Starting from a random spin configurations if not provided
    if S is None:
        S = random.rand(n,3)-0.5
        S = S / np.sqrt(np.sum(S**2, 1))[:,np.newaxis] # normalize to unit length
    else:
        n = S.shape[0]

    acc = 0 # Accumulating the num of acceptance 
    for i in range(nMC):
        for k in range(n): # Iterate over every spins and calculated mean field created by its two nearest neighbours (periodic boundary condition
            if k==0:
                F = J* (S[n-1,:] + S[1,:]) - H 
            elif k==n-1:
                F = J* (S[n-2,:] + S[0,:]) - H
            else:
                F = J* (S[k-1,:] + S[k+1,:]) - H
            
            # Propose a new spin direction
            S_new = random.rand(3)-0.5
            S_new = S_new / np.sqrt(np.sum(S_new**2))
            dE = np.dot(S_new-S[k,:], F) 
            # Accept/reject: Metropolis–Hastings algorithm
            if random.rand(1)[0] < np.exp(-dE/T):
                S[k,:] = S_new
                acc += 1
    if debug:
        print('Accept rate is: {:1.2f}'.format(acc/(nMC*n)), 'at T= {:1.5f}'.format(T))
    return S    

def MC_t_fast(J, H, T, S=None, n=20, nMC=1000, debug=False):
    """
    Non-interacting spins are updated simultaneously， utilizing the advantage of internal parallel computing for matrix manipulation.
    Here the non-interacting spins are all the spins wiht even/odd index.
    
    Inputs:
    J: exchange interaction between nearest neigbor spins
    H: magnetic field: 1 by 3
    T: temperature relative to J
    S: num_of_spins by 3
    n: num_of_spins
    nMC：num of MC steps per spin
    Output: a spin configuration
    """
    
    if S is None:
        S = random.rand(n,3)-0.5
        S = S / np.sqrt(np.sum(S**2, 1))[:,np.newaxis]
    else:
        n = S.shape[0] 

    acc = 0
    for i in range(nMC):
        # Preparing nearest neighbours
        S_rollU = np.roll(S,-1, axis=0) # next
        S_rollD = np.roll(S, 1, axis=0) # last
        
        # Update spins of even/odd index simutaniusly (parallel computing)
        for j in [0,1]:
            F = J * (S_rollU[j::2] + S_rollD[j::2]) - H[np.newaxis,:]
            n0 = len(F)
            S_new = random.rand(n0,3)-0.5
            S_new = S_new / np.sqrt(np.sum(S_new**2,axis=1))[:,np.newaxis]
            
            dE = np.sum((S_new-S[j::2])*F, axis=1) 
            idx = random.rand(n0) < np.exp(-dE/T)
            
            S[j::2,:][idx] = S_new[idx]
            acc += np.sum(idx)
            
    if debug:
        print('Accept rate is: {:1.2f}'.format(acc/(nMC*n)), 'at T= {:1.5f}'.format(T))
    return S  

def anneal(J=1, H=np.array([0,0,0]), initT=1, endT=0.1, coolR=0.92, S=None, n=20, nMC=1000,debug=True):
    """
    Simulated annealing from hight temperature Ts[0] to Ts[-1].
    J: exchange interaction between nearest neigbor spins
    initT: staring high temperature in unit of J
    endT:  the lowest temperature in unit of J
    coolR: cool rate (<1)
    S: num_of_spins by 3
    n: num_of_spins
    nMC：num of MC steps per spin for every temperature step
    """
    # Estimate the time needed
    nSteps = np.int(np.log(endT/initT)/np.log(coolR) +1)
    
    tic = time.time()
    S = MC_t_fast(J, H, initT, S=S, n=n, nMC=nMC,debug=False)
    toc = time.time()
        
    print(nSteps, 'anneal steps;', 'Time per step: {:1.0f} s;'.format(toc-tic), 'Total time needed: {:1.0f} s'.format((toc-tic)*nSteps))
    
    T = initT*coolR
    
    # Annealing loop
    i = 1
    while T>endT:
        print('Anneal Step No. ', i)
        i += 1
        # Note: taking the state of the last T as the start for the current MC
        S = MC_t_fast(J, H, T, S=S, n=n, nMC=nMC, debug=debug)
        T = T * coolR
        
    return S, T


In [3]:
n = 100 # nunber of spins on the chain
J = -1. # nearest-neighbour echange interactions
H = np.array([0,0.1,0]) # magnetic field
initT=0.5*np.abs(J)
endT=0.05*np.abs(J)
coolR = 0.9

np.int(np.log(endT/initT)/np.log(coolR)+1)

22

In [4]:
S, T = anneal(J=J, initT=initT, endT=endT, coolR=coolR, S=None, n=n, nMC=2000)

S = MC_t_fast(J, H, endT, S=S, n=n, nMC=5000, debug=True) # more equillibrum steps at the lowest T

22 anneal steps; Time per step: 0 s; Total time needed: 5 s
Anneal Step No.  1
Accept rate is: 0.38 at T= 0.45000
Anneal Step No.  2
Accept rate is: 0.34 at T= 0.40500
Anneal Step No.  3
Accept rate is: 0.29 at T= 0.36450
Anneal Step No.  4
Accept rate is: 0.25 at T= 0.32805
Anneal Step No.  5
Accept rate is: 0.22 at T= 0.29525
Anneal Step No.  6
Accept rate is: 0.18 at T= 0.26572
Anneal Step No.  7
Accept rate is: 0.16 at T= 0.23915
Anneal Step No.  8
Accept rate is: 0.14 at T= 0.21523
Anneal Step No.  9
Accept rate is: 0.12 at T= 0.19371
Anneal Step No.  10
Accept rate is: 0.10 at T= 0.17434
Anneal Step No.  11
Accept rate is: 0.09 at T= 0.15691
Anneal Step No.  12
Accept rate is: 0.08 at T= 0.14121
Anneal Step No.  13
Accept rate is: 0.07 at T= 0.12709
Anneal Step No.  14
Accept rate is: 0.07 at T= 0.11438
Anneal Step No.  15
Accept rate is: 0.06 at T= 0.10295
Anneal Step No.  16
Accept rate is: 0.06 at T= 0.09265
Anneal Step No.  17
Accept rate is: 0.05 at T= 0.08339
Anneal Step No

## Plot the ground state spin structure

In [7]:
# Spin positions: chain along the x direction
X = np.vstack([np.arange(n)-n/2, np.zeros(n), np.zeros(n)]).T
spinL = 1 # plot spin length
atomR = 1 # plot atom radius
cylR = 0.005 # plot bond thinckness

try:
    scene.delete()
except:
    pass
    
scene = vp.canvas(title='MagStr', width=1200, height=100,x=500,y=500, center=vp.vector(0,0,0), background=vp.color.black,exit=True)

for i in range(n):
    vp.arrow(pos=vp.vector(*(X[i]-spinL*S[i]/2)), axis=vp.vector(*(spinL*S[i]))) # spins
    vp.sphere(pos=vp.vector(*X[i]), color=vp.color.orange, radius=atomR*0.1) # atoms 

for i in range(n-1):
    vp.cylinder(pos=vp.vector(*(X[i])), axis=vp.vector(*(X[i+1]-X[i])), radius=cylR, color=vp.color.gray(0.5))
 

<IPython.core.display.Javascript object>

# Molecular dynamcis

In [8]:
# Deriv calculation 
def deriv(J, H, S):
    """
    Deriv dS/dt calculation
    J: exchange interation constant
    H: 1 by 3 array for magnetic field
    S: num_of_spins by 3 for a spin configuration
    """
    n = S.shape[0]
    Fs = np.zeros_like(S)
    for k in range(n):
            if k==0:
                Fs[k] = J* (S[n-1,:] + S[1,:]) - H
            elif k==n-1:
                Fs[k] = J* (S[n-2,:] + S[0,:]) - H
            else:
                Fs[k] = J* (S[k-1,:] + S[k+1,:]) - H
    return np.cross(S, Fs)

def deriv_fast(J, H, S):
    """
    Deriv dS/dt calculation
    J: exchange interation constant
    H: 1 by 3 array for magnetic field
    S: num_of_spins by 3 for a spin configuration
    """
    n = S.shape[0]   
    F = J*(np.roll(S,-1, axis=0) + np.roll(S, 1, axis=0)) -H[np.newaxis, :]
    return np.cross(S, F)

# Numerical integration 
def Runge_Kutta(func, x0, Ndt=100, dt=0.01):
    """
    func: function to calculate the deriv
    x0: starting point
    Ndt: num of time stamps
    dt: time step size
    
    Return: y at different times and an array of time stamps
    """

    ts = np.zeros(Ndt)
    x = x0
    y = np.zeros(np.hstack([Ndt, x0.shape]))
    y[0] = x0
    
    print(y.shape)
    for i in range(1, Ndt):
        DD_1= func(x)*dt
        DD_2= func(x+DD_1/2)*dt
        DD_3= func(x+DD_2/2)*dt
        DD_4= func(x+DD_3)*dt

        # Spin configuration at t+dt
        ts[i] = ts[i-1] + dt
        x = x + 1/6*(DD_1 + 2*DD_2 + 2*DD_3 + DD_4)

        y[i] = x 
    return y, ts

# Dynamicsal
def dynamics_FT1d(St, ts, xs, qs, omega):
    """
    Temporal and spacial Fourier transformation for a one-dimensional spin chain.
    Input:
    St: n_times by num_of_spins by 3 array for spin configurations (num_of_spins by 3) at differt times 
    ts: 1d array for the n_times time stamps
    xs: 1d array for the n spin positions
    omega: 1d array for energies to calculated
    """
    
    qphase = np.exp(1j *2*np.pi*np.matmul(qs,xs)) # phase factor due to different locations
    ophase = np.exp(-1j*np.matmul(ts.reshape([-1,1]), omega)) # phase factor due to different time
       
    sxqw = np.matmul(np.matmul(qphase, St[:,:,0].T), ophase) 
    syqw = np.matmul(np.matmul(qphase, St[:,:,1].T), ophase)
    szqw = np.matmul(np.matmul(qphase, St[:,:,2].T), ophase)
    
    return np.absolute(sxqw**2+syqw**2+szqw**2)

In [9]:
xs= np.arange(n).reshape([1,-1])
qs = np.linspace(1./n, 1, num=n-1, endpoint=False).reshape([-1,1])
omega = np.linspace(0, 5*np.abs(J), num=50, endpoint=False).reshape([1,-1])

St, ts = Runge_Kutta(lambda S: deriv_fast(J, H, S), S, Ndt=3000, dt=0.05)
print(St.shape)

(3000, 100, 3)
(3000, 100, 3)


## Plot the procession of spins

In [10]:
try:
    scene1.delete()
except:
    pass
    
scene1 = vp.canvas(title='MagStr', width=1300, height=300,x=500,y=500, center=vp.vector(0,0,0), background=vp.color.white, exit=True)
scene1.camera.pos=vp.vector(*([0,0,2]))

mid = np.floor_divide(n,2)
which_spin = np.arange(mid-2, mid+2+1)

for idx, i in enumerate(which_spin):
    vp.sphere(pos=vp.vector(*X[i]), color=vp.color.orange, radius=atomR*0.1) # atoms
    if idx<len(which_spin)-1:
        pointer = vp.cylinder(pos=vp.vector(*X[i]), axis=vp.vector(*(X[i+1]-X[i])), radius=cylR, color=vp.color.black)

which_time = range(0,500,1)
color = [colors.to_rgba(c)
          for c in cm.get_cmap('Reds')(which_time /np.max(which_time ))]

# Plot spins at different times in a loop; the time stamp is encoded by the arrow color
for idx, i in  enumerate(which_time ):
    for j in which_spin:
        vp.arrow(pos=vp.vector(*(X[j]-0*spinL*St[i,j,:]/2)), 
                 axis=vp.vector(*(0.5*spinL*St[i,j,:])), 
                 color=vp.vector(*color[idx][:3]), round=True, shaftwidth=0.01*spinL, headwidth=0.02*spinL) 
#scene1.capture('cones.png')

<IPython.core.display.Javascript object>

In [11]:
try:
    scene2.delete()
except:
    pass
    
scene2 = vp.canvas(title='MagStr', width=1000, height=300,x=500,y=500, center=vp.vector(0,0,0), background=vp.color.white, exit=True)
#scene1.camera.pos=vp.vector(0,0,1)
mid = np.floor_divide(n,2)
which_spin = np.arange(mid-2, mid+2+1)

for idx, i in enumerate(which_spin):
    vp.sphere(pos=vp.vector(*X[i]), color=vp.color.orange, radius=atomR*0.1) # atoms
    if idx<len(which_spin)-1:
        pointer = vp.cylinder(pos=vp.vector(*X[i]), axis=vp.vector(*(X[i+1]-X[i])), radius=cylR, color=vp.color.black)

which_time = range(0,400,2)
color = [colors.to_rgba(c)
          for c in cm.get_cmap('Reds')(which_time /np.max(which_time ))]

arrows = []

for i in  [0]:
    for j in which_spin:
        arrows.append(vp.arrow(pos=vp.vector(*(X[j]-0*spinL*St[i,j,:]/2)), axis=vp.vector(*(0.5*spinL*St[i,j,:])), 
                               round=True, shaftwidth=0.01*spinL, headwidth=0.02*spinL))#, make_trail=True, trail_type="points", interval=10, retain=50) )
        vp.sphere(pos=vp.vector(*(X[j]-0*spinL*St[i,j,:]/2))+vp.vector(*(0.5*spinL*St[i,j,:])), radius=0.01*atomR)

i = 1
while i<len(which_time)-5:
    vp.sleep(0.1)
    for idx, j in enumerate(which_spin):
        arrows[idx].axis = vp.vector(*(0.5*spinL*St[which_time[i],j,:]))
        vp.sphere(pos=vp.vector(*(X[j] + spinL*St[which_time[i],j,:]/2)), radius=0.01*atomR, color=vp.vector(*color[i][:3]))
    i +=1
    #scene2.capture('spins_t.png')


<IPython.core.display.Javascript object>

KeyboardInterrupt: 

In [17]:
import imageio
images = []

path = r'D:\Downloads/'
pattern = 'spins_t({:1.0f}).png'

for i in np.arange(1,194):
    fn = path + pattern.format(i)
    images.append(imageio.imread(fn))
imageio.mimsave(path+'/movie.gif', images, format='GIF', fps=5)

In [93]:
imageio.help(name='gif')

GIF-PIL - Static and animated gif (Pillow)

    A format for reading and writing static and animated GIF, based
    on Pillow.

    Images read with this format are always RGBA. Currently,
    the alpha channel is ignored when saving RGB images with this
    format.

    Parameters for reading
    ----------------------
    None

    Parameters for saving
    ---------------------
    loop : int
        The number of iterations. Default 0 (meaning loop indefinitely).
    duration : {float, list}
        The duration (in seconds) of each frame. Either specify one value
        that is used for all frames, or one value for each frame.
        Note that in the GIF format the duration/delay is expressed in
        hundredths of a second, which limits the precision of the duration.
    fps : float
        The number of frames per second. If duration is not given, the
        duration for each frame is set to 1/fps. Default 10.
    palettesize : int
        The number of colors to quantize t

## Dynamical correlation functions (scattering spectrom)

In [20]:
sqw = dynamics_FT1d(St, ts, xs, qs, omega)

In [21]:

plt.figure(figsize=(5,4))
Xax, Yax = np.meshgrid(qs, omega)
plt.pcolor(Xax, Yax, sqw.T,vmin=0.0,vmax=2000000)
plt.xlabel(r'$Q$')
plt.ylabel(r'$\hbar\omega$')
plt.ylim([0,4])

# Quantum linear spin wave theory 
plt.gca().plot(qs, -2*J*(1 - np.cos(2*np.pi*qs))+np.sqrt(np.sum(H**2)),c='w', label='Quantum theory')
plt.legend()
#plt.savefig(r'D:\Downloads/spin_dispersion_fm.png')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-21-c17fd13e2d0f>:3: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  plt.pcolor(Xax, Yax, sqw.T,vmin=0.0,vmax=2000000)
